In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
df = pd.read_csv('../data/cleaned_data.csv')


# Convert columns to datetime objects
df['Event Date'] = pd.to_datetime(df['Event Date'], format='%Y %B %d')
df['Event Time'] = pd.to_datetime(df['Event Time'].astype(str)).dt.time


# Extract features from Event Date
df['Month'] = df['Event Date'].dt.month
df['Day'] = df['Event Date'].dt.day
df['Weekday'] = df['Event Date'].dt.day_name()

# Create season feature (winter vs non-winter)
df['Season'] = df['Month'].apply(lambda m: 'winter' if m in [12, 1, 2] else 'non-winter')

# Convert Event Time to extract the hour (assuming it's in 24-hour format)
# Since 'Event Time' is already datetime.time, you can extract hour directly:
df['Hour'] = df['Event Time'].apply(lambda t: t.hour) #Using apply to access the hour attribute

# Categorize time of day
def categorize_time(hour):
    if 5 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 17:
        return 'afternoon'
    elif 17 <= hour < 21:
        return 'evening'
    else:
        return 'night'

df['Time Of Day'] = df['Hour'].apply(categorize_time)

# Create rush hour flag
def is_rush_hour(hour):
    return 1 if (7 <= hour < 10 or 16 <= hour < 19) else 0

df['Rush Hour'] = df['Hour'].apply(is_rush_hour) #Removed extra =

# Select columns to display for checking
columns = ['Event Date', 'Event Time', 'Hour', 'Time Of Day', 'Rush Hour', 'Season', 'Weekday', 'Day', 'Month']
print(df[columns].head())

# Store the new dataset to a new CSV file
df = pd.read_csv('../data/feature_engineered_data.csv', index=False)

/Users/chukkalok/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


  event_date event_time  hour time_of_day  rush_hour      season    weekday  \
0 2024-11-30   08:46:00     8     morning          1  non-winter   Saturday   
1 2024-11-30   19:09:00    19     evening          0  non-winter   Saturday   
2 2024-11-29   20:30:00    20     evening          0  non-winter     Friday   
3 2024-11-29   13:03:00    13   afternoon          0  non-winter     Friday   
4 2024-11-27   15:20:00    15   afternoon          0  non-winter  Wednesday   

   day  month  
0   30     11  
1   30     11  
2   29     11  
3   29     11  
4   27     11  


In [2]:
# enhance the column uace code by using external census data in 2010 
import pandas as pd
# load census data
df_census_data = pd.read_csv('../data/census_data_2010.csv')

In [12]:
# Store the new dataset to a new CSV file
df = pd.read_csv('../data/feature_engineered_data.csv')

In [24]:
# add one feature Density in df_census_data 
# Arealand: unit meters square
# Density = POP/(AREALAND/1000)
df_census_data['Density'] = df_census_data['POP']/df_census_data['AREALAND']*1000

# normalize the columns
df_census_data['Density'] = (df_census_data['Density'] - df_census_data['Density'].min()) / (df_census_data['Density'].max() - df_census_data['Density'].min())

#select uace and density and show head
df_census_data[['UACE', 'Density']].head()

,UACE,Density
0,37,0.144370
1,64,0.086678
2,91,0.160758
3,118,0.131157
4,145,0.173549


In [25]:
# join df with df_census_data 
# join on df[Primary UZA UACE Code] = df_census_data[UACE]
# left join df on df_census_data
# select only Density from df_census_data and keep all columns in df


# Assuming df and df_census_data are already defined DataFrames

# Select only the 'UACE' and 'Density' columns from df_census_data
df_census_data_subset = df_census_data[['UACE', 'Density']]

# Perform a left merge on the key: df["Primary UZA UACE Code"] = df_census_data["UACE"]
df = pd.merge(df, df_census_data_subset, left_on='Primary UZA UACE Code', right_on='UACE', how='left')

# df now contains all columns from df and the 'Density' column from df_census_data
print(df.head())



   NTD ID  Primary UZA UACE Code Rail/Bus/Ferry        Mode Name Mode TOS  \
0   20080                  63217            Bus              Bus   MB  DO   
1   40035                  65863            Bus  Demand Response   DR  DO   
2   50027                  57628           Rail       Light Rail   LR  DO   
3   90026                  78661           Rail       Light Rail   LR  DO   
4   20080                  63217            Bus              Bus   MB  DO   

   Year  Event Date Event Time                           Event Type  ...  \
0  2024  2024-11-30   08:46:00                   Non-Rail Collision  ...   
1  2024  2024-11-30   19:09:00                   Non-Rail Collision  ...   
2  2024  2024-11-29   20:30:00  Homicide not against Transit Worker  ...   
3  2024  2024-11-29   13:03:00   Assault not against Transit Worker  ...   
4  2024  2024-11-27   15:20:00                   Non-Rail Collision  ...   

  Total Serious Injuries Month  Day    Weekday      Season  Hour  Time Of Day  \

In [26]:
# Analyze missing value distribution and patterns
#count any missing values
df.isnull().sum()

NTD ID                                           0
Primary UZA UACE Code                            0
Rail/Bus/Ferry                                   0
Mode Name                                        0
Mode                                             0
TOS                                              0
Year                                             0
Event Date                                       0
Event Time                                       0
Event Type                                       0
Event Type Group                                 0
Safety/Security                                  0
Total Injuries                                   0
Total Fatalities                                 0
Towed (Y/N)                                      0
Number of Transit Vehicles Involved              0
Number of Non-Transit Vehicles Involved          0
Number of Cars on Involved Transit Vehicles      0
Location Type                                    0
Total Serious Injuries         

In [27]:
#print df columns name
print(df.columns)

Index(['NTD ID', 'Primary UZA UACE Code', 'Rail/Bus/Ferry', 'Mode Name',
       'Mode', 'TOS', 'Year', 'Event Date', 'Event Time', 'Event Type',
       'Event Type Group', 'Safety/Security', 'Total Injuries',
       'Total Fatalities', 'Towed (Y/N)',
       'Number of Transit Vehicles Involved',
       'Number of Non-Transit Vehicles Involved',
       'Number of Cars on Involved Transit Vehicles', 'Location Type',
       'Total Serious Injuries', 'Month', 'Day', 'Weekday', 'Season', 'Hour',
       'Time Of Day', 'Rush Hour', 'UACE', 'Density'],
      dtype='object')


In [28]:
# compare rows that has missing values in density, and those with values, group by year
# count the rows with missing values, group by year
missing_rows = df[df['Density'].isnull()].groupby('Year').size()

# print out side by side, one column being missing rows count, the other column being row without missing values, group by year
print(pd.concat([missing_rows, df.groupby('Year').size()], axis=1))

       0      1
Year           
2014  52   6556
2015  92   8999
2016  79   9260
2017  91   9145
2018  76   9475
2019  97   9818
2020  53   7144
2021  74   8276
2022  60   9582
2023  68  10365
2024  82   9991


In [30]:
# calculate statistics 
df[df['Density'].isnull()]

# top ten uace code 
df[df['Density'].isnull()].groupby('Primary UZA UACE Code').size().sort_values(ascending=False).head(10)

Primary UZA UACE Code
39889    545
9536     114
22612     64
42        18
17        16
71060     16
50533     14
61948     13
99999     10
67140      5
dtype: int64

Reference for the found UACE, Area Name and population on https://www.transportation.gov/grants/infra-urban-and-rural-areas
| UACE  | Urban Area Name                                | Population |
|-------|------------------------------------------------|------------|
| 39889 | Honolulu, HI                                   | 853,252    |
| 09536 | Bradenton–Sarasota–Venice, FL                    | 779,075    |
| 22612 | Daytona Beach–Palm Coast–Port Orange, FL         | 402,126    |
| 50533 | Livermore–Pleasanton–Dublin, CA                  | 240,381    |
| 67140 | Palmdale–Lancaster, CA                           | 359,559    |


In [31]:
# create a column has_fatalities if total_fatalities is greater than 0
df['Has Fatalities'] = df['Total Fatalities'].apply(lambda x: 1 if x > 0 else 0)

df['Has Fatalities'].value_counts()

0    95579
1     3032
Name: Has Fatalities, dtype: int64

In [33]:
# count if density has missing values, how many rows has fatalities = 1
df[df['Density'].isnull()].groupby('Has Fatalities').size()

Has Fatalities
0    806
1     18
dtype: int64

In [34]:
# update the feature_engineered_data.csv with df
df.to_csv('../data/feature_engineered_data.csv', index=False)

In [36]:
df_test = pd.read_csv('../data/feature_engineered_data.csv')
df_test.head()

,NTD ID,Primary UZA UACE Code,Rail/Bus/Ferry,Mode Name,Mode,TOS,Year,Event Date,Event Time,Event Type,...,Month,Day,Weekday,Season,Hour,Time Of Day,Rush Hour,UACE,Density,Has Fatalities
0,20080,63217,Bus,Bus,MB,DO,2024,2024-11-30,08:46:00,Non-Rail Collision,...,11,30,Saturday,non-winter,8,morning,1,63217.0,0.513374,0
1,40035,65863,Bus,Demand Response,DR,DO,2024,2024-11-30,19:09:00,Non-Rail Collision,...,11,30,Saturday,non-winter,19,evening,0,65863.0,0.224168,0
2,50027,57628,Rail,Light Rail,LR,DO,2024,2024-11-29,20:30:00,Homicide not against Transit Worker,...,11,29,Friday,non-winter,20,evening,0,57628.0,0.231116,1
3,90026,78661,Rail,Light Rail,LR,DO,2024,2024-11-29,13:03:00,Assault not against Transit Worker,...,11,29,Friday,non-winter,13,afternoon,0,78661.0,0.380573,0
4,20080,63217,Bus,Bus,MB,DO,2024,2024-11-27,15:20:00,Non-Rail Collision,...,11,27,Wednesday,non-winter,15,afternoon,0,63217.0,0.513374,0
